In [7]:
import os
import openai
#occasionally there are issues with the key using llamaindex and Ragas so both methods are set for the key. 
OPENAI_API_KEY="sk-xxxxxxx"
openai.api_key=OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = "xxxxxx"

from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader("input files")
documents = loader.load()


In [8]:
for document in documents:
    document.metadata['filename'] = document.metadata['source']

In [9]:

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 


# Assuming you have already set up the necessary API keys or authentication methods for OpenAI
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model='gpt-4')
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Create a TestsetGenerator instance with the specified generator, critic, and embeddings
generator = TestsetGenerator.from_langchain(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings
)


# Generate the test set with specified distribution among evolution types
testset = generator.generate_with_langchain_docs(
    documents=documents, 
    test_size=12, 
    raise_exceptions=False,
    distributions={
        simple: 0.5, 
        reasoning: 0.25, 
        multi_context: 0.25
    }
)



embedding nodes:   0%|          | 0/174 [00:00<?, ?it/s]

Generating:   0%|          | 0/12 [00:00<?, ?it/s]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [10]:
testdf = testset.to_pandas()



In [11]:

from llama_index.core import ( VectorStoreIndex, SimpleDirectoryReader, Response,
                               )
import pandas as pd
import nest_asyncio

nest_asyncio.apply()
pd.set_option("display.max_colwidth", 0)
# Initialize the SimpleDirectoryReader with directory path
reader = SimpleDirectoryReader(input_dir="input Files")

# Load the documents
documents = reader.load_data()


# Create the index from the documents
index = VectorStoreIndex.from_documents(documents)

print(f"Loaded {len(documents)} docs")

for document in documents:
    # Check if 'file_name' key exists in the metadata dictionary
    if "file_name" in document.metadata:
        # Change 'file_name' to 'filename'
        document.metadata["filename"] = document.metadata.pop("file_name")

from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# build index
index = VectorStoreIndex.from_documents(documents)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

# list of queries
query_list = testdf['question'].tolist()
    


# dictionary to store queries and responses
query_response_dict = {}

for query in query_list:
    response = query_engine.query(query)
    query_response_dict[query] = response

for query, response in query_response_dict.items():
    print(f'Query: {query}\nResponse: {response}\n')




/Users/joshuaschoen/PycharmProjects/RagProject/.venv/lib/python3.12/site-packages/pydantic/v1/config.py:183: RuntimeWarning: coroutine 'Executor.wrap_callable_with_index.<locals>.wrapped_callable_async' was never awaited
  return type('Config', base_classes, namespace)
Task was destroyed but it is pending!
task: <Task pending name='Task-68' coro=<as_completed.<locals>.sema_coro() running at /Users/joshuaschoen/PycharmProjects/RagProject/.venv/lib/python3.12/site-packages/ragas/executor.py:37> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/tasks.py:618]>
Task was destroyed but it is pending!
task: <Task pending name='Task-12' coro=<as_completed.<locals>.sema_coro() running at /Users/joshuaschoen/PycharmProjects/RagProject/.venv/lib/python3.12/site-packages/ragas/executor.py:38> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[as_completed.<locals>._on_completio

Loaded 7 docs
Query: What rights do the Investors have to participate in future equity offerings?
Response: Investors have the right of first offer to participate in future equity offerings. The Company must first offer any new securities to each Major Investor before offering them to others. Major Investors can elect to purchase a portion of these new securities based on the proportion of Common Stock they hold compared to the total Common Stock outstanding. If any Major Investor does not fully exercise their right to purchase, other Fully Exercising Investors have the opportunity to purchase the remaining unsubscribed shares.

Query: What is the definition of an accredited investor as defined in Rule 501(a) of Regulation D promulgated under the Securities Act?
Response: An individual who has a net worth exceeding $1 million (either individually or jointly with their spouse) at the time of the purchase, or has an income exceeding $200,000 in each of the two most recent years (or joint

In [12]:
query_response_tuples = [(query, response) for query, response in query_response_dict.items()]
responses_df = pd.DataFrame(query_response_tuples, columns=["question", 'answer'])


In [13]:
result = testdf.merge(responses_df, on='question', how='left')


In [15]:
#turns the resulting joined dataframe into a csv.
result.to_csv("your_directory", index=False)